# Problem Sheet Day 1

_Gdansk Summer School 'Picturing Quantum Weirdness 2023_


In this problem sheet, we will get the hang of working with the basic generators of circuits and ZX-diagrams concretely, using matrix calculations. For this, we will use the `sympy` library.

In [1]:
from sympy import *
from sympy.physics.quantum import TensorProduct
from fractions import Fraction

def T(*args):
    if len(args) == 1: return args[0]
    elif len(args) == 0: return Matrix([[1]])
    else: return TensorProduct(args[0], T(*args[1:]))

alpha = var("alpha")
beta = var("beta")
gamma = var("gamma")

With this library, we can construct matrices with `Matrix`. Then `*` is matrix multiplication, and `T` is tensor product.

Note `T` takes any number of arguments, e.g. $A \otimes B \otimes C$ is `T(A,B,C)`, and if you want to be fancy: $A^{\otimes n}$ is `T(*n*[A])`. (Python trivia: Why does that work??)

In [2]:
M = Matrix([[1,  2],
            [3,  4]])
display(M)
display(M * M * M)
display(T(M,M))

Matrix([
[1, 2],
[3, 4]])

Matrix([
[37,  54],
[81, 118]])

Matrix([
[1,  2,  2,  4],
[3,  4,  6,  8],
[3,  6,  4,  8],
[9, 12, 12, 16]])

We can also define some variables with `var` which can be used in mathmatical expressions, and substituted via `.subs(...)`. Variable names can be any string, but `sympy` knows how to pretty-print some variable names, e.g. Greek letters.

NOTE: $\sqrt{2}$ is `sqrt(2)`, $i$ is `I`, $\pi$ is `pi`, and $e^x$ is `exp(x)`, so phases $e^{i \alpha}$ are written `exp(i * alpha)`.

In [3]:
alpha = var("alpha")
phase = exp(I * alpha)
epi4 = exp(I * pi / 4)

display(alpha)
display(phase)
display(phase.subs(alpha, -pi / 2))
display(phase.subs(alpha, pi / 4) == epi4)
display(re(epi4) + I * im(epi4))

alpha

exp(I*alpha)

-I

True

sqrt(2)/2 + sqrt(2)*I/2

That should be enough to get going. If in doubt, [read the docs](https://docs.sympy.org/latest/index.html).

# Question 0

First some basics. Define matrices for:
 * `z0` := $|0\rangle$, `z1` := $|1\rangle$, `x0` := $|{+}\rangle$, `x1` := $|{-}\rangle$
 * `bz0` := $\langle 0|$, `bz1` := $\langle 1|$, `bx0` := $\langle {+}|$, `bx1` := $\langle {-}|$
 * `w` for the 2D identity matrix ("wire")
 * `swap` for the swap

Compute various products and tensor products and show the results are sensible.

In [4]:
z0 = Matrix([[1],
             [0]])
z1 = Matrix([[0],
             [1]])
x0 = Matrix([[1],
             [1]]) / sqrt(2)
x1 = Matrix([[1],
             [-1]]) / sqrt(2)
bz0 = z0.T
bz1 = z1.T
bx0 = x0.T
bx1 = x1.T

w = Matrix([[1, 0],
            [0, 1]])

swap = Matrix([[1, 0, 0, 0],
               [0, 0, 1, 0],
               [0, 1, 0, 0],
               [0, 0, 0, 1]])

display(bz0)
display(z0)

Matrix([[1, 0]])

Matrix([
[1],
[0]])

# Question 1

Define a function that produces the matrix of a Z-spider. It should take 3 arguments: `m` for input legs, `n` for output legs, and `phase` for phase. The phase should have a default value of 0.

Build this function in (at least) 2 different ways:
 1. by building the $2^n \times 2^m$ matrix of the spider explicitly (call this function `zs`)
 2. by using sums, compositions, and tensor products of the generators from the previous question

Test your implementation by comparing the matrices for various choices of inputs, outputs, and phases. (Don't forget to check no-legged spiders!)

In [19]:
def zs(m: int, n: int, phase: float = 0.0):
    """
    Produces the matrix of a Z spider.

    :param m: The input legs
    :param n: The output legs
    :param phase: Phase of the spider
    :return: A matrix of the Z spider
    """
    mat = zeros(2**m, 2**n)
    mat[0,0] = 1
    mat[2**m - 1,2**n - 1] += exp(I * phase)
    return mat

def zspider(m: int, n: int, phase: float = 0.0):
    """
    Produces the matrix of a Z spider using tensor products.

    :param m: The input legs
    :param n: The output legs
    :param phase: Phase of the spider
    :return: A matrix of the Z spider
    """
    zero_proj = T(*m*[z0]) * T(*n*[bz0])
    phase_proj = T(*m*[z1]) * T(*n*[bz1]) * exp(I * phase)
    return zero_proj + phase_proj

display(zs(3, 2, pi / 3))

Matrix([
[1, 0, 0,           0],
[0, 0, 0,           0],
[0, 0, 0,           0],
[0, 0, 0,           0],
[0, 0, 0,           0],
[0, 0, 0,           0],
[0, 0, 0,           0],
[0, 0, 0, exp(I*pi/3)]])

# Question 2

Similarly, define a function for X-spiders:

1. by first defining the Hadamard gate `had` and using `zs` (call this function `xs`)
2. using the generators from Question 0

Again, check these two definitions agree for various numbers of legs and angles.

In [26]:
had = 1/sqrt(2) * Matrix([[1,  1],
                           [1, -1]])

def xs(m: int, n: int, phase: float = 0.0):
    """
    Produces the matrix of a X spider.

    :param m: The input legs
    :param n: The output legs
    :param phase: Phase of the spider
    :return: A matrix of the X spider
    """
    return T(*m*[had]) * zs(m, n, phase) * T(*n*[had])

def xsipder(m: int, n: int, phase: float = 0.0):
    """
    Produces the matrix of a X spider using tensor products.

    :param m: The input legs
    :param n: The output legs
    :param phase: Phase of the spider
    :return: A matrix of the X spider
    """
    plus_proj = T(*m*[x0]) * T(*n*[bx0])
    minus_proj = T(*m*[x1]) * T(*n*[bx1]) * exp(I * phase)
    return plus_proj + minus_proj

display(xs(3,2,pi / 3))
display(xsipder(3,2,pi / 3))

Matrix([
[sqrt(2)/8 + sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 - sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 - sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 + sqrt(2)*exp(I*pi/3)/8],
[sqrt(2)/8 - sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 + sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 + sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 - sqrt(2)*exp(I*pi/3)/8],
[sqrt(2)/8 - sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 + sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 + sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 - sqrt(2)*exp(I*pi/3)/8],
[sqrt(2)/8 + sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 - sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 - sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 + sqrt(2)*exp(I*pi/3)/8],
[sqrt(2)/8 - sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 + sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 + sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 - sqrt(2)*exp(I*pi/3)/8],
[sqrt(2)/8 + sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 - sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 - sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 + sqrt(2)*exp(I*pi/3)/8],
[sqrt(2)/8 + sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 - sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 - sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 + sqrt(2)*exp(I*pi/3)/8

Matrix([
[sqrt(2)/8 + sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 - sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 - sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 + sqrt(2)*exp(I*pi/3)/8],
[sqrt(2)/8 - sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 + sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 + sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 - sqrt(2)*exp(I*pi/3)/8],
[sqrt(2)/8 - sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 + sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 + sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 - sqrt(2)*exp(I*pi/3)/8],
[sqrt(2)/8 + sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 - sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 - sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 + sqrt(2)*exp(I*pi/3)/8],
[sqrt(2)/8 - sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 + sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 + sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 - sqrt(2)*exp(I*pi/3)/8],
[sqrt(2)/8 + sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 - sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 - sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 + sqrt(2)*exp(I*pi/3)/8],
[sqrt(2)/8 + sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 - sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 - sqrt(2)*exp(I*pi/3)/8, sqrt(2)/8 + sqrt(2)*exp(I*pi/3)/8

# Question 3

Verify the correctness of the Euler decomposition of the Hadamard. Find the right scalar factor so the matrices of the LHS and RHS agree exactly.

**Hint:** If you have some variables, you may need to call the `simplify(...)` function before comparing matrices. If a constant phase is stuck and won't simplify any more, try splitting the real and imaginary parts and putting back together, i.e. `expr1 = re(expr) + I * im(expr)`.

In [51]:
theta = var("theta")
euler_decomp = zs(1,1,pi / 2) * xs(1,1,pi / 2) * zs(1,1,pi / 2) * exp(I * pi / 4) * (- sqrt(2) * I)

display(simplify(re(euler_decomp) + I * im(euler_decomp)))

Matrix([
[1,  1],
[1, -1]])

# Question 4

* Make an XOR using an X spider and a scalar multiplication, and show it acts like XOR on $\{|0\rangle, |1\rangle\}$.
* Make a CNOT and TONC (a CNOT with a control on the second qubit and target on the first) using Z spiders, X spiders, swaps, and scalar multiplication. Check that CNOT acts as expected on basis states and `swap * CNOT * swap` is the same as `TONC`
* Check XOR is associative and `CNOT * TONC * CNOT = swap`

In [77]:
print("XOR")

XOR = xs(2, 0, pi)

for st in [
    T(bz0, bz0),
    T(bz0, bz1),
    T(bz1, bz0),
    T(bz1, bz1),
]:
    print("Input:", st)
    display(st * XOR)

print("CNOT")

CNOT = T(zs(1,2), eye(2)) * T(eye(2), xs(2,1)) * sqrt(2)

display(CNOT)

print("TONC")

TONC = swap * CNOT * swap

display(TONC)

print("CNOT * TONC * CNOT")

display(CNOT * TONC * CNOT)




XOR
Input: Matrix([[1, 0, 0, 0]])


Matrix([[0]])

Input: Matrix([[0, 1, 0, 0]])


Matrix([[1]])

Input: Matrix([[0, 0, 1, 0]])


...

Matrix([
[1, 0, 0, 0],
[0, 1, 0, 0],
[0, 0, 0, 1],
[0, 0, 1, 0]])

TONC


Matrix([
[1, 0, 0, 0],
[0, 0, 0, 1],
[0, 0, 1, 0],
[0, 1, 0, 0]])

CNOT * TONC * CNOT


Matrix([
[1, 0, 0, 0],
[0, 0, 1, 0],
[0, 1, 0, 0],
[0, 0, 0, 1]])